In [0]:
%scala

case class DeviceIoTData (battery_level: Long, c02_level: Long, 
    cca2: String, cca3: String, cn: String, device_id: Long, 
    device_name: String, humidity: Long, ip: String, latitude: Double,
    lcd: String, longitude: Double, scale:String, temp: Long, timestamp: Long)

defined class DeviceIoTData

In [0]:
%scala

val ds = spark.read.json("/FileStore/iot_devices.json").as[DeviceIoTData]

ds.printSchema

ds.show(5, false)

root
-- battery_level: long (nullable = true)
-- c02_level: long (nullable = true)
-- cca2: string (nullable = true)
-- cca3: string (nullable = true)
-- cn: string (nullable = true)
-- device_id: long (nullable = true)
-- device_name: string (nullable = true)
-- humidity: long (nullable = true)
-- ip: string (nullable = true)
-- latitude: double (nullable = true)
-- lcd: string (nullable = true)
-- longitude: double (nullable = true)
-- scale: string (nullable = true)
-- temp: long (nullable = true)
-- timestamp: long (nullable = true)

+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
battery_level|c02_level|cca2|cca3|cn |device_id|device_name |humidity|ip |latitude|lcd |longitude|scale |temp|timestamp |
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
8 |868 |US |USA |United States|1 |meter-gauge-1xbYRYcj |51 |68.161.225.1 |38.0 |green |-97.0 |Celsius|34 |1458444054093|
7 |1473 |NO |NOR |Norway |2 |sensor-pad-2n2Pea |70 |213.161.254.1|62.47 |red |6.15 |Celsius|11 |1458444054119|
2 |1556 |IT |ITA |Italy |3 |device-mac-36TWSKiT |44 |88.36.5.1 |42.83 |red |12.83 |Celsius|19 |1458444054120|
6 |1080 |US |USA |United States|4 |sensor-pad-4mzWkz |32 |66.39.173.154|44.06 |yellow|-121.32 |Celsius|28 |1458444054121|
4 |931 |PH |PHL |Philippines |5 |therm-stick-5gimpUrBB|62 |203.82.41.9 |14.58 |green |120.97 |Celsius|25 |1458444054122|
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
only showing top 5 rows

ds: org.apache.spark.sql.Dataset[DeviceIoTData] = [battery_level: bigint, c02_level: bigint ... 13 more fields]

In [0]:
%scala
// Detect failing devices with battery levels below a threshold.

ds.select($"battery_level", $"c02_level", $"device_name").where($"battery_level" < 8).sort($"c02_level").show(5, false)

+-------------+---------+---------------------------+
battery_level|c02_level|device_name |
+-------------+---------+---------------------------+
6 |800 |sensor-pad-165544zEXVK1ep9 |
4 |800 |meter-gauge-1700815tyVmZ |
1 |800 |sensor-pad-167460xU6np7Gum8|
4 |800 |meter-gauge-159899xYtKf |
3 |800 |sensor-pad-167632hnUNk |
+-------------+---------+---------------------------+
only showing top 5 rows

In [0]:
%scala
//Identify offending countries with high levels of CO2 emissions.

val newDS = ds
  .filter(d => {d.c02_level > 1300})
  .groupBy($"cn")
  .avg()
  .sort($"avg(c02_level)".desc)

newDS.show(10, false)


+------------------------------+------------------+--------------+--------------+-------------+-------------+------------------+---------+------------------+
cn |avg(battery_level)|avg(c02_level)|avg(device_id)|avg(humidity)|avg(latitude)|avg(longitude) |avg(temp)|avg(timestamp) |
+------------------------------+------------------+--------------+--------------+-------------+-------------+------------------+---------+------------------+
Solomon Islands |3.0 |1588.0 |187433.0 |40.0 |-9.43 |159.95 |21.0 |1.458444060894E12 |
Federated States of Micronesia|3.0 |1573.0 |78806.0 |55.0 |6.92 |158.25 |13.0 |1.45844405755E12 |
Rwanda |2.5 |1560.5 |102085.0 |44.0 |-2.0 |30.0 |21.5 |1.458444058393E12 |
British Indian Ocean Territory|7.0 |1560.0 |83930.0 |54.0 |-6.0 |71.5 |27.0 |1.458444057649E12 |
Aruba |4.0 |1559.0 |99899.0 |84.0 |12.51 |-70.0 |19.5 |1.458444057927E12 |
Isle of Man |5.0 |1548.0 |137958.5 |46.5 |54.23 |-4.57 |24.5 |1.4584440593755E12|
Gambia |3.0 |1544.5 |34748.0 |66.0 |13.47 |-16.57 |16.0 |1.458444056277E12 |
Lesotho |2.5 |1537.5 |74949.0 |72.0 |-29.41 |27.990000000000002|21.0 |1.458444057568E12 |
Cuba |5.2 |1534.8 |66819.2 |42.0 |21.444 |-79.336 |26.6 |1.4584440572212E12|
Gabon |8.0 |1523.0 |106953.0 |30.0 |0.38 |9.45 |28.0 |1.45844405847E12 |
+------------------------------+------------------+--------------+--------------+-------------+-------------+------------------+---------+------------------+
only showing top 10 rows

newDS: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cn: string, avg(battery_level): double ... 7 more fields]

In [0]:
%scala
//Compute the min and max values for temperature, battery level, CO2, and humidity.

import org.apache.spark.sql.functions._ 

ds.select(min("temp"), max("temp"), min("humidity"), max("humidity"), min("c02_level"), max("c02_level"), min("battery_level"), max("battery_level")).show(10)

+---------+---------+-------------+-------------+--------------+--------------+------------------+------------------+
min(temp)|max(temp)|min(humidity)|max(humidity)|min(c02_level)|max(c02_level)|min(battery_level)|max(battery_level)|
+---------+---------+-------------+-------------+--------------+--------------+------------------+------------------+
 10| 34| 25| 99| 800| 1599| 0| 9|
+---------+---------+-------------+-------------+--------------+--------------+------------------+------------------+

import org.apache.spark.sql.functions._

In [0]:
%scala
//Sort and group by average temperature, CO2, humidity, and country.

ds.filter(d => {d.temp > 25 && d.humidity > 75})
  .select("temp", "humidity", "cn")
  .groupBy($"cn")
  .avg()
  .sort($"avg(temp)".desc, $"avg(humidity)".desc).as("avg_humidity").show(10, false)


+----------------------+---------+-------------+
cn |avg(temp)|avg(humidity)|
+----------------------+---------+-------------+
Monaco |34.0 |91.0 |
Anguilla |34.0 |83.0 |
British Virgin Islands|34.0 |81.0 |
Turkmenistan |34.0 |80.0 |
Suriname |34.0 |79.0 |
Gibraltar |34.0 |78.0 |
Liechtenstein |34.0 |76.0 |
Vanuatu |33.5 |84.0 |
Cameroon |33.0 |91.0 |
Fiji |33.0 |78.0 |
+----------------------+---------+-------------+
only showing top 10 rows